In [ ]:
#import all libraries to be used in the exercise
import numpy as np
import pandas as pd
import os
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
pd.set_option('display.max_columns', None)


In [ ]:
#show the files in the folder I´m working in. This will help me to check that I have the right files to load.
print(os.listdir()) 

In [ ]:
#Upload the files
df_flights = pd.read_csv('Customer Flight Activity.csv' , sep=',')
df_loyalty = pd.read_csv('Customer Loyalty History.csv' , sep=',')

In [ ]:
#Show the construccion of each dataframe
print(df_flights.info())
print(df_loyalty.info())

In [ ]:
#Verify the index of the dataframes is a custom index or a predefined numeric index
print(df_flights.index)
print(df_loyalty.index)

In [ ]:
#Displays the first few rows of the dataframes so its structure can be checked
display(df_flights.head())
display(df_loyalty.head())

In [ ]:
#Check if the columns names are correct
print(df_flights.columns)
print(df_loyalty.columns)

In [ ]:
#Standardise column names in both dataframes, remove blank spaces, convert everything to lower case and replace spaces with underscores
df_flights.columns = df_flights.columns.str.strip().str.lower().str.replace(" ", "_")
df_loyalty.columns = df_loyalty.columns.str.strip().str.lower().str.replace(" ", "_")

print(df_flights.columns)
print(df_loyalty.columns)

In [ ]:
#Show how many unique values there are in each column of the dataframes
print("Number of unique values in Customer Flight Analysis:")
print(df_flights.nunique())

print("Number of unique values in Customer Loyalty History:")
print(df_loyalty.nunique())

In [ ]:
print("unique values in columm year:", df_flights['year'].unique())
print("unique values in columm month:", df_flights['month'].unique())

In [ ]:
#Check all the duplicated rows and count how many there are
print(df_flights.duplicated().sum())   #means that there are 1864 duplicated rows que the same values.
print(df_loyalty.duplicated().sum())

In [ ]:
df_flights[df_flights.duplicated(keep=False)].head(50)

In [ ]:
#Remove duplicates that are exactly the same, removing the extra copies if all columns are the same
#and reset the index to avoid gaps after removing duplicates
df_flights.drop_duplicates(inplace=True)
df_flights.reset_index(drop=True, inplace=True)


In [ ]:
#Run this code to check that the duplicates have been removed correctly. If it returns 0, there are no exact matches.
print(df_flights.duplicated().sum())


In [ ]:
#To manage duplicates with differences in some columns, we search for records that have the same loyalty_number, year and month, 
#but with different values in other columns.
df_flights[df_flights.duplicated(subset=['loyalty_number', 'year', 'month'], keep=False)]

In [ ]:
#Merge duplicate values
df_flights = df_flights.groupby(['loyalty_number', 'year', 'month']).agg({
    "flights_booked": "sum",
    "flights_with_companions": "sum",
    "total_flights": "sum",
    "distance": "sum",
    "points_accumulated": "sum",
    "points_redeemed": "sum",
    "dollar_cost_points_redeemed": "sum"}).reset_index()


In [ ]:
#Select another option, like keeping only the rows with the most points. The code will be:
#df_flights = df_flights.sort_values(by='points_accumulated', ascending=False).drop_duplicates(subset=['loyalty_number', 'year', 'month'], keep='first')


In [ ]:
print(df_flights.info())

In [ ]:
#I am interested in seeing the unique values to test my hypothesis, even though we already know that there are no duplicates in the customer loyalty history.
text_columns = df_loyalty.select_dtypes(include=['object']).columns
for column_object in text_columns:
    print(f"The singles values of the column {column_object} are:")
    print(df_loyalty[column_object].unique())

In [ ]:
#The country column has only one value, which does not matter for me, so I will delete it.
df_loyalty.drop(columns=['country'], inplace=True)
print(df_loyalty.columns)

In [ ]:
#To avoid errors in further analysis, I will normalise columns with categorical values: gender, education, marital_status, loyalty_card and enrolment_type.
columns_to_standar = ['gender','education', 'marital_status', 'loyalty_card', 'enrollment_type']
df_loyalty[columns_to_standar] = df_loyalty[columns_to_standar].apply(lambda x: x.str.strip().str.lower())


In [ ]:
for column in columns_to_standar:
    print(f"The singles values normalise of the column {column} are: {df_loyalty[column].unique()}")

In [ ]:
#Once we have finished handling duplicates, we move on to managing nulls. 
#We have already seen that customer flight activity has no nulls, so we focus on customer loyalty history.
#First, we must see where there are nulls so we can decide how to deal with them.
df_loyalty.isnull().sum()


In [ ]:
#We can calculate the percentage or make a bar chart to get an idea of the amount of nulls in the columns involved.
columns_with_nulls = ['salary', 'cancellation_year', 'cancellation_month']
nulls_ratio = np.round(df_loyalty[columns_with_nulls].isnull().mean()*100, 2)
print(nulls_ratio)

In [ ]:
plt.figure(figsize=(6, 4))
sns.barplot(x=nulls_ratio.index, y=nulls_ratio.values)
plt.ylabel("% of null values")
plt.title("Map of null values in Salary, Cancellation Year and Cancellation Month")
plt.show()

In [ ]:
#We can also run some basic statistics to better understand how the data's distributed or to help identify problems.
df_loyalty[['salary', 'cancellation_year', 'cancellation_month']].describe()

In [ ]:
columns_with_nulls = ['salary', 'cancellation_year', 'cancellation_month']
plt.figure(figsize=(6, 4))
sns.heatmap(df_loyalty[columns_with_nulls].isnull(), cmap='Blues', cbar=False, yticklabels=False)
plt.title("Map of null values in Salary, Cancellation Year and Cancellation Month")
plt.show()

Cada barra de representa a un cliente,en oscuro con valor nulo y el claro con valor real. Salary tiene los valores nulos muy dispersos, por lo que no podemos considerar que sigan un patrón

In [ ]:
#Planteamos la hipotesis de que los clientes con valores nulos en cancellation year y cancellation month no son errores, si no que son clientes que siguen siendo activos en el porgrama de fidelidad.
cancelled_clients = df_loyalty[['cancellation_year', 'cancellation_month']].notnull().all(axis=1).value_counts()
plt.figure(figsize=(6,4))
plt.bar(["nulls", "active"], cancelled_clients.values)
plt.xlabel("membership")
plt.ylabel("number of members")
plt.show()



In [ ]:
plt.figure(figsize=(6, 4))
sns.boxplot(x=df_loyalty['cancellation_year'].isnull(), y=df_loyalty['clv'])
plt.xticks([0, 1], ["unsubscrib members", "Active members"])
plt.ylabel("Customer Lifetime Value (CLV)")
plt.xlabel("Membership Status")
plt.title("Distribution of CLV between active and deleted customers")
plt.show()


In [ ]:
#mean_clv_cancelled = df_loyalty[df_loyalty[['cancellation_year', 'cancellation_month']].notnull().all(axis=1)]['clv'].mean()

#mean_clv_active = df_loyalty[df_loyalty[['cancellation_year', 'cancellation_month']].isnull().any(axis=1)]['clv'].mean()

#mean_clv_cancelled, mean_clv_active

(8131.776768263183, 7968.7647402862995)

In [ ]:
#We replace null values with 0 to avoid problems with further analysis. Always remember that 0 means that the client is still active.
df_loyalty['cancellation_year'] = df_loyalty['cancellation_year'].fillna(0)
df_loyalty['cancellation_month'] = df_loyalty['cancellation_month'].fillna(0)


In [ ]:
#We check that the replacements have been executed correctly.
df_loyalty['cancellation_year'] = df_loyalty['cancellation_year'].astype(int)
df_loyalty['cancellation_month'] = df_loyalty['cancellation_month'].astype(int)
df_loyalty[['cancellation_year', 'cancellation_month']].info()
df_loyalty[['cancellation_year', 'cancellation_month']].isnull().sum()

In [ ]:
negative_salaries = (df_loyalty['salary'] < 0).sum()
print(negative_salaries)

In [ ]:
df_loyalty[df_loyalty['salary'] < 0]

In [ ]:
df_loyalty.loc[df_loyalty['salary'] < 0, 'salary'] = np.nan

In [ ]:
new_negative_salaries = (df_loyalty['salary'] < 0).sum()
print(new_negative_salaries)

In [ ]:

plt.figure(figsize=(8, 5))
plt.hist(df_loyalty['salary'].dropna(), bins=30, edgecolor='black', alpha=0.7, log=True)
plt.title("Distribución de Salary")
plt.xlabel("Salary")
plt.ylabel("Frecuencia")
plt.show()


In [ ]:
plt.figure(figsize=(6, 4))
sns.boxplot(x=df_loyalty['salary'])
plt.xlabel("Salary")
plt.title("Possible outliers in salary distribution")
plt.show()


In [ ]:
#To see how we can manage the null values in the salary column, we are going to make a series of comparisons with columns in the same table. 
#This will show us their relationship and whether they follow a pattern.
plt.figure(figsize=(8, 5))
sns.boxplot(x='education', y='salary', data=df_loyalty)
plt.xticks(rotation=45)
plt.title("Distribution of Salary by Education Level")
plt.show()



In [ ]:
plt.figure(figsize=(12, 6))
sns.boxplot(x='city', y='salary', data=df_loyalty)
plt.xticks(rotation=90)
plt.title("Salary Distribution by City ")
plt.show()


In [ ]:
plt.figure(figsize=(24, 6))
sns.boxplot(x='postal_code', y='salary', data=df_loyalty)
plt.xticks(rotation=90)
plt.title("Salary Distribution by Postal Code ")
plt.show()


In [ ]:
plt.figure(figsize=(6, 4))
sns.scatterplot(x='salary', y='clv', data=df_loyalty, alpha=0.5)
plt.xlabel("Salary")
plt.ylabel("CLV")
plt.title("Salary and CLV ratios")
plt.show()


In [ ]:
plt.figure(figsize=(6, 4))
sns.barplot(x='enrollment_type', y='salary', data=df_loyalty, estimator=np.median)
plt.title("Median Salary by Membership")
plt.show()


In [ ]:
#To substitute missing values we will use IterativeImputer as it is the most complete method to replace null values.
#These models cannot use categorical values so we will map relevant columns such as education or postal code to numerical ones before executing this technique.
education_mapping ={'High School or Below': 0,
                    'college': 1,
                    'bachelor': 2,
                    'master': 3,
                    'doctor': 4}
df_loyalty['education_number'] = df_loyalty['education'].map(education_mapping)

df_loyalty['postal_code_number'], _ = df_loyalty['postal_code'].factorize()

In [ ]:
df_loyalty[['education', 'education_number', 'postal_code', 'postal_code_number']].head(10)

In [ ]:
print(df_loyalty.columns)


In [ ]:
df_loyalty[['salary', 'clv']].describe()

In [ ]:

colums_imputer = ['salary', 'clv', 'education_number', 'postal_code_number']
imputer = IterativeImputer(max_iter=100, random_state=42)
df_loyalty[colums_imputer] = imputer.fit_transform(df_loyalty[colums_imputer])

In [ ]:
df_loyalty['salary'].isnull().sum()


In [ ]:
df_loyalty[['salary', 'clv']].describe()

In [ ]:
#we remove the columns that we have converted to numerical columns as we no longer need them.
df_loyalty.drop(columns=['education_number', 'postal_code_number'], inplace=True)


In [ ]:
#We will join the dataframes using the loyalty_number column. 
#We want to keep all the clients of df_loyalty and add the data of df_flights. So we will make a merge left join.
df_merged = df_loyalty.merge(df_flights, on='loyalty_number', how="left")

In [ ]:
df_merged.info()

In [ ]:
df_merged.sample(20)

In [ ]:
#Export the new clean dataset to a CSV file
df_merged.to_csv("airline_loyalty_programme.csv", index=False)
print(os.listdir()) 

In [ ]:
#there are columns that for the visualisation phase we don't care if they are joined, so let's proceed to join them.
df_merged['enrollment_date'] = pd.to_datetime(df_merged['enrollment_year'].astype(str) + '-' + df_merged['enrollment_month'].astype(str) + '-01', format="%Y-%m-%d")
df_merged =df_merged.drop(columns=['enrollment_year', 'enrollment_month'])
col_position = df_merged.columns.get_loc('cancellation_year')
df_merged.insert(col_position, 'enrollment_date', df_merged.pop('enrollment_date'))
df_merged = df_merged.set_index('loyalty_number')
df_merged.sample(15)



### ❓ Question 1: Distribution of the number of flights booked per month during each year

In [ ]:
flights_month_and_year =df_flights.groupby(['year', 'month'])['flights_booked'].sum().reset_index()
print(flights_month_and_year)

In [ ]:
flights_month_and_year =df_flights.groupby(['year', 'month'])['flights_booked'].sum().reset_index()
plt.figure(figsize=(10,4))
sns.lineplot(x='month', y='flights_booked', hue='year', data=flights_month_and_year, marker="o", linewidth=2, markersize=6)
plt.title("Distribution of flights booked per month and year")
plt.xticks(ticks=range(1, 13))
plt.xlabel("Month")
plt.ylabel("Flights booked")
plt.legend(title='Year')
plt.grid(True, alpha=0.5)


In [ ]:
flights_month_and_year =df_flights.groupby(['year', 'month'])['flights_booked'].sum().reset_index()
plt.figure(figsize=(10,4))
sns.barplot(x='month', y='flights_booked', hue='year', data=flights_month_and_year)
plt.title("Distribution of flights booked per month and year")
plt.xticks(ticks=range(1, 13))
plt.xlabel("Month")
plt.ylabel("Flights booked")
plt.legend(title='Year')
plt.grid(True, alpha=0.5)

### ❓ Question 2: Is there a connection between flight distance and points accumulated?

In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(x="distance", y="points_accumulated", hue="loyalty_card", data= df_merged, marker="o", palette="rocket")
plt.xlabel("Flight´s distance")
plt.ylabel("Accumulated points")
plt.title("Ratio between flight distance and accumulated points")
plt.grid(True)


In [ ]:
sns.lmplot(x="distance", y="points_accumulated", hue="loyalty_card", data=df_merged, markers="o", height=6, aspect=1.5, palette="rocket", scatter_kws={"alpha": 0.3, "s": 50}, line_kws={"linewidth":3})
plt.xlabel("Flight´s distance")
plt.ylabel("Accumulated points")
plt.title("Ratio between flight distance and accumulated points")

In [ ]:
plt.figure(figsize=(10, 6))
sns.violinplot(x="distance", y="points_accumulated", hue="loyalty_card", data= df_merged, marker="o", palette="dark:#5A9_r")
plt.xlabel("Flight´s distance")
plt.ylabel("Accumulated points")
plt.title("Ratio between flight distance and accumulated points")

### ❓ Question 3: What is the distribution of clients by province and state?

In [ ]:
members_province = df_loyalty["province"].value_counts()
print(members_province)


In [ ]:
plt.figure(figsize=(12, 6))
members_province.plot(kind="bar", color="pink", edgecolor="purple", linewidth=3)
plt.xlabel("Province")
plt.xticks(rotation=45)
plt.ylabel("Number of clients")
plt.title("Ratio clients by province")
plt.grid(True, alpha=0.5)


In [ ]:

members_percentage = (members_province / members_province.sum()) * 100
members_percentage = members_percentage.sort_values(ascending=True)
plt.figure(figsize=(10, 6))
ax = sns.barplot(x=members_percentage, y=members_percentage.index, palette="Purples_r")
ax.bar_label(ax.containers[0], fmt="%.1f%%", padding=5, fontsize=10)

plt.xlabel("Percentage of Clients")
plt.ylabel("Province")
plt.title("Percentage of Clients by Province")
plt.xlim(0, max(members_percentage) + 5) 
plt.grid(axis="x", linestyle="--", alpha=0.5)



### ❓ Question 4: Compare the average salary by education level

In [ ]:
salary_by_education = df_loyalty.groupby("education")["salary"].mean().round(2).sort_values()
print(salary_by_education)


In [ ]:
df_salary = salary_by_education.reset_index() 
plt.figure(figsize=(10, 6))
ax = sns.barplot(data=df_salary, x="education", y="salary", color="#6C8E68", edgecolor="#8e686e", linewidth=5)
ax.bar_label(ax.containers[0], fmt="%.2f", fontsize=10, color="black", padding=5)
plt.xlabel("Education")
plt.ylabel("Average Salary")
plt.title("A Comparison of Average Salaries by Educational Level")
plt.grid(axis="y", linestyle="--", alpha=0.7)
plt.xticks(rotation=45)


In [ ]:

sns.boxplot(x=df_loyalty["education"], y=df_loyalty["salary"], 
            boxprops={"facecolor": "#6C8E68", "edgecolor": "#8E686E", "linewidth": 2},
            medianprops={"color": "#403033", "linewidth": 2},  
            whiskerprops={"color": "#8E686E", "linewidth": 2}, 
            capprops={"color": "#8E686E", "linewidth": 2},
            flierprops={"marker": "o", "color": "#8E686E", "alpha": 0.5})  
plt.xlabel("Educational Attainment")
plt.ylabel("Income")
plt.title("Income and Educational Attainment Structure")
plt.grid(True, alpha=0.5, linestyle="--")
plt.xticks(rotation=45)





In [ ]:
#We are going to see the same graph only with the college data in order to see it properly.
df_college = df_loyalty[df_loyalty["education"] == "college"]
plt.figure(figsize=(6, 6))
sns.boxplot(y=df_college["salary"], 
            boxprops={"facecolor": "#6C8E68", "edgecolor": "#8E686E", "linewidth": 2},
            medianprops={"color": "#403033", "linewidth": 2},
            whiskerprops={"color": "#8E686E", "linewidth": 2},  # Bigotes
            capprops={"color": "#8E686E", "linewidth": 2},
            flierprops={"marker": "o", "markerfacecolor": "#8E686E", "alpha": 0.5})
plt.ylabel("Income")
plt.title("Income Distribution for College Education Level")
plt.grid(True, alpha=0.5, linestyle="--")

In [ ]:

df_loyalty["education"] = df_loyalty["education"].str.strip().str.lower()
plt.figure(figsize=(10, 8))  
sns.histplot(data=df_loyalty, y="salary", hue="education", bins=40, kde=True, alpha=0.3, palette="mako", edgecolor="black", multiple="layer", stat="count")
plt.ylabel("Salary")
plt.xlabel("Number of clients")
plt.title("Salary by educational level")
plt.legend(title="educational level", labels=df_loyalty["education"].unique(), loc="upper right")
plt.grid(True, alpha=0.5, linestyle="--")

### ❓ Question 5: What is the percentage of clients for each type of loyalty card?


In [ ]:
loyalty_counts = df_loyalty["loyalty_card"].value_counts(normalize=True) * 100
print(loyalty_counts)


In [ ]:
plt.figure(figsize=(7, 5))
colors = ["#dbb770", "#db7094", "#db8270", "gold"]
wedges, texts, autotexts = plt.pie(loyalty_counts, labels=loyalty_counts.index, autopct="%1.1f%%", colors=colors, startangle=140, wedgeprops={"edgecolor": "#f1debd", "linewidth": 2.5})
for text, color in zip(texts, colors):
    text.set_text(text.get_text().upper()) 
    text.set_color(color) 
    text.set_fontsize(12) 
plt.title("Customer Distribution by Type of Loyalty Card", pad=30)
plt.axis("equal")

In [ ]:

df = pd.read_csv('airline_loyalty_programme.csv' , sep=',')

# Confirmar que los datos se han cargado correctamente

df.head(5)


In [ ]:
numerical_variables = df.select_dtypes(include="number").columns
correlation_data = df[numerical_variables].corr()
plt.figure(figsize=(12, 8))
sns.heatmap(correlation_data, annot=True, cmap='flare', fmt='.2f', linewidths=.5)
plt.title('Correlation Between the Variables', pad=20, fontsize=14)
plt.xticks(rotation=45, ha="right") 
plt.yticks(rotation=0)


### ❓ Question 6: How are clients distributed according to marital status and gender?

In [ ]:

marital_gender_counts = df_loyalty.groupby(["marital_status", "gender"]).size().unstack()
print(marital_gender_counts)


In [ ]:
plt.figure(figsize=(6, 4))
marital_gender_counts.plot(kind="bar", figsize=(10, 6), color=["#c7c4ff", "#c4dfff"], edgecolor="#ffc7c4")
plt.xlabel("Marital Status")
plt.ylabel("Number of clients")
plt.title("Distribution of Clients by Marital Status and Gender")
plt.xticks(rotation=45)
plt.legend(title="Gender")
plt.grid(axis="y", linestyle="--", alpha=0.7)


In [ ]:
marital_counts = df_loyalty["marital_status"].value_counts()
plt.figure(figsize=(6, 4))
explode = [0.05] * len(marital_counts)
wedges, texts, autotexts = plt.pie(marital_counts, labels=marital_counts.index, autopct="%1.1f%%", colors=plt.cm.Pastel2.colors, startangle=140, explode=explode)
for text, color in zip(texts, colors):
    text.set_text(text.get_text().upper()) 
    text.set_color(color) 
    text.set_fontsize(12) 
plt.title("Customers by Marital Status", pad=30)
plt.axis("equal")